# Phyllosphere of Hawaiian Plants - 16S Analysis

Note that the sequence files and the qiime2 files containing all the sequences (demux-paired-end.qza) and the trimmed ITS sequences (trimmed.qza) are not provided inn the repository because of their size. The steps below allow you to recreate the files.

## Import Sequences into Qiime2

The sequences for 16S amplicon sequencing of the 47 samples should be downloaded from GenBank's SRA database under Bioproject PRJNA880325. The titles of Biosamples containing the 16S reads start with "16S".

Once downloaded the files should be moved to a directory called "Data" within the current directory.

In [ ]:
!mkdir 01-ProcessReads
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path 00-illumina/data \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path 01-ProcessReads/demux-paired-end.qza

In [ ]:
!qiime demux summarize \
 --i-data 01-ProcessReads/demux-paired-end.qza \
 --p-n 100000 \
 --verbose \
 --o-visualization 01-ProcessReads/demux.qzv

## Filter reads and Remove Primers from reads:

In [ ]:
!qiime cutadapt trim-paired \
  --i-demultiplexed-sequences 01-ProcessReads/demux-paired-end.qza \
  --p-cores 12 \
  --p-front-f NTGYCAGCMGCCGCGGTAA \
  --p-front-r NCGYCAATTYMTTTRAGTTT \
  --p-discard-untrimmed \
  --o-trimmed-sequences 01-ProcessReads/trimmed-seqs.qza \
  --verbose > 01-ProcessReads/trim.log

In [ ]:
!qiime demux summarize \
 --i-data 01-ProcessReads/trimmed-seqs.qza \
 --p-n 100000 \
 --verbose \
 --o-visualization 01-ProcessReads/trimmed-seqs.qzv

In [ ]:
!qiime dada2 denoise-paired \
  --i-demultiplexed-seqs 01-ProcessReads/trimmed-seqs.qza \
  --p-trim-left-f 0 \
  --p-trim-left-r 0 \
  --p-trunc-len-f 280 \
  --p-trunc-len-r 200 \
  --p-n-threads 10 \
  --o-representative-sequences 01-ProcessReads/rep-seqs-dada.qza \
  --o-table 01-ProcessReads/table-dada.qza \
  --o-denoising-stats 01-ProcessReads/stats-dada.qza


In [ ]:
!qiime metadata tabulate \
  --m-input-file 01-ProcessReads/stats-dada.qza \
  --o-visualization 01-ProcessReads/stats-dada.qzv

!qiime feature-table summarize \
  --i-table 01-ProcessReads/table-dada.qza \
  --o-visualization 01-ProcessReads/table-dada.qzv \
  --m-sample-metadata-file map.txt

## Download SILVA database

!mkdir silva

!wget -O silva/silva-138-99-seqs.qza https://data.qiime2.org/2020.11/common/silva-138-99-seqs.qza

!wget -O silva/silva-138-99-tax.qza https://data.qiime2.org/2020.11/common/silva-138-99-tax.qza    

## Get SILVA classifier

Downloaded from: https://data.qiime2.org/2021.4/common/silva-138-99-nb-classifier.qza


In [ ]:
!mkdir silva

!wget -O silva/silva-138-99-seqs.qza https://data.qiime2.org/2020.11/common/silva-138-99-seqs.qza
!wget -O silva/silva-138-99-tax.qza https://data.qiime2.org/2020.11/common/silva-138-99-tax.qza    

!wget -O silva/silva-138-99-nb-classifier.qza https://data.qiime2.org/2020.11/common/silva-138-99-nb-classifier.qza

## Assign taxonomy

In [ ]:
!qiime feature-classifier classify-consensus-vsearch \
  --i-query 01-ProcessReads/rep-seqs-dada.qza \
  --i-reference-reads silva/silva-138-99-seqs.qza \
  --i-reference-taxonomy silva/silva-138-99-tax.qza \
  --p-threads 30 \
  --o-classification 01-ProcessReads/taxonomy-vsearch.qza \
  --verbose

!qiime metadata tabulate \
  --m-input-file 01-ProcessReads/taxonomy-vsearch.qza \
  --o-visualization 01-ProcessReads/taxonomy-vsearch.qzv

## Create tree of ASVs

In [ ]:
!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences 01-ProcessReads/rep-seqs-dada.qza \
  --o-alignment 01-ProcessReads/aligned-rep-seqs.qza \
  --o-masked-alignment 01-ProcessReads/masked-aligned-rep-seqs.qza \
  --o-tree 01-ProcessReads/unrooted-tree.qza \
  --o-rooted-tree 01-ProcessReads/rooted-tree.qza

# Analyses

In [ ]:
!qiime diversity core-metrics-phylogenetic \
  --i-table 01-ProcessReads/table-dada.qza \
  --p-sampling-depth 8000 \
  --i-phylogeny 01-ProcessReads/rooted-tree.qza \
  --m-metadata-file map.txt \
  --output-dir 02-Analyses/core-metrics-results-vsearch

# Taxa plots
!qiime taxa barplot \
  --i-table 01-ProcessReads/table-dada.qza \
  --i-taxonomy 01-ProcessReads/taxonomy-vsearch.qza \
  --m-metadata-file map.txt \
  --o-visualization 02-Analyses/taxa-bar-plots-vsearch.qzv